<a href="https://colab.research.google.com/github/Vervion/MonoGS/blob/main/MonoGSResults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1) Clone your fork AND pull the submodules in one go
!git clone --recursive https://github.com/Vervion/MonoGS.git
%cd MonoGS

In [1]:
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:07
🔁 Restarting kernel...


: 

In [1]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [4]:
!pip install \
    numpy==1.26.4 \
    glfw \
    imgviz \
    opencv-python \
    PyOpenGL \
    PyGLM \
    configargparse \
    munch \
    lpips \
    open3d \
    seaborn \
    matplotlib \
    scikit-learn \
    pandas \
    evo \
    wandb

INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 133.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 103.3 M

In [21]:
%cd /content/MonoGS
!pip install plyfile
!pip install rich
!pip install trimesh
!pip install torchmetrics

/content/MonoGS
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 31.4 MB/s eta 0:00:00


In [1]:
!pip install ninja

/content/MonoGS/submodules/simple-knn
running build_ext
W1203 21:41:29.344000 11715 site-packages/torch/utils/cpp_extension.py:630] Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
W1203 21:41:29.366000 11715 site-packages/torch/utils/cpp_extension.py:521] The detected CUDA version (12.5) has a minor version mismatch with the version that was used to compile PyTorch (12.8). Most likely this shouldn't be a problem.
W1203 21:41:29.367000 11715 site-packages/torch/utils/cpp_extension.py:531] There are no g++ version bounds defined for CUDA version 12.5
building 'simple_knn._C' extension
creating build
creating build/temp.linux-x86_64-cpython-311
gcc -pthread -B /usr/local/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /usr/local/include -fPIC -O2 -isystem /usr/local/include -fPIC -I/usr/local/lib/python3.11/site-packages/torch/include -I/usr/local/lib/python3.11/site-packages/torch/include/t

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
%cd /content/MonoGS/submodules/simple-knn

# Clean any failed build
!rm -rf build dist simple_knn.egg-info

# Build extension and install in editable mode
!python setup.py build_ext --inplace
!pip install -e . --no-build-isolation


In [3]:
#Patch simple-knn cuda
%cd /content/MonoGS/submodules/simple-knn

# Insert "#include <cfloat>" after the first line that starts with "#include"
txt = open("simple_knn.cu").read()
if "FLT_MAX" in txt and "<cfloat>" not in txt and "<float.h>" not in txt:
    lines = txt.splitlines()
    for i, line in enumerate(lines):
        if line.startswith("#include"):
            lines.insert(i+1, "#include <cfloat>  // added for FLT_MAX")
            break
    open("simple_knn.cu", "w").write("\n".join(lines))
    print("Patched simple_knn.cu")
else:
    print("No patch needed or already patched")


/content/MonoGS/submodules/simple-knn
Patched simple_knn.cu


In [5]:
%cd /content/MonoGS/submodules/diff-gaussian-rasterization
!rm -rf build dist diff_gaussian_rasterization.egg-info

!python setup.py build_ext --inplace
!pip install -e . --no-build-isolation

/content/MonoGS/submodules/diff-gaussian-rasterization
running build_ext
W1203 21:59:55.157000 16689 site-packages/torch/utils/cpp_extension.py:521] The detected CUDA version (12.5) has a minor version mismatch with the version that was used to compile PyTorch (12.8). Most likely this shouldn't be a problem.
W1203 21:59:55.157000 16689 site-packages/torch/utils/cpp_extension.py:531] There are no g++ version bounds defined for CUDA version 12.5
building 'diff_gaussian_rasterization._C' extension
creating /content/MonoGS/submodules/diff-gaussian-rasterization/build
creating /content/MonoGS/submodules/diff-gaussian-rasterization/build/temp.linux-x86_64-cpython-311
creating /content/MonoGS/submodules/diff-gaussian-rasterization/build/temp.linux-x86_64-cpython-311/cuda_rasterizer
[1/5] /usr/local/cuda/bin/nvcc --generate-dependencies-with-compile --dependency-output /content/MonoGS/submodules/diff-gaussian-rasterization/build/temp.linux-x86_64-cpython-311/cuda_rasterizer/forward.o.d -I/usr/

In [6]:
#Check imports
%cd /content/MonoGS

code = """
import simple_knn, diff_gaussian_rasterization
print('simple_knn:', simple_knn.__file__)
print('diff_gaussian_rasterization:', diff_gaussian_rasterization.__file__)
"""
!python - <<"$code"

/content/MonoGS
/bin/bash: line 5: warning: here-document at line 5 delimited by end-of-file (wanted `
import simple_knn, diff_gaussian_rasterization
print('simple_knn:', simple_knn.__file__)
print('diff_gaussian_rasterization:', diff_gaussian_rasterization.__file__)
')


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
cp -r /content/drive/MyDrive/Datasets/tum/rgbd_dataset_freiburg1_desk.tgz /content/MonoGS/datasets


In [12]:
tar -xvzf your_filename.tgz

NameError: name 'unzip' is not defined

In [24]:
!pip uninstall -y evo
!pip install evo==1.13.0

Found existing installation: evo 1.13.0
Uninstalling evo-1.13.0:
  Successfully uninstalled evo-1.13.0
  Using cached evo-1.13.0-py3-none-any.whl


In [35]:
%cd /content/MonoGS
!python slam.py --config configs/rgbd/tum/fr1_desk.yaml

/content/MonoGS
MonoGS: saving results in results/MonoGS_datasets/2025-12-03-23-34-53
error: XDG_RUNTIME_DIR not set in the environment.
FEngine (64 bits) created at 0x7bff6c821010 (threading is enabled)
FEngine resolved backend: OpenGL
Failed to open X display. (exiting).
MonoGS: Resetting the system
MonoGS: Initialized map
MonoGS: Evaluating ATE at frame:  23
[evaluate_evo] Skipping in-code evo evaluation (will do it offline with evo CLI).
MonoGS: Evaluating ATE at frame:  52
[evaluate_evo] Skipping in-code evo evaluation (will do it offline with evo CLI).
MonoGS: Resetting the opacity of non-visible Gaussians
MonoGS: Evaluating ATE at frame:  75
[evaluate_evo] Skipping in-code evo evaluation (will do it offline with evo CLI).
MonoGS: Evaluating ATE at frame:  99
[evaluate_evo] Skipping in-code evo evaluation (will do it offline with evo CLI).
MonoGS: Evaluating ATE at frame:  120
[evaluate_evo] Skipping in-code evo evaluation (will do it offline with evo CLI).
MonoGS: Evaluating ATE

KeyboardInterrupt: 

In [36]:
import os, glob, json

run_dir = "/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53"  # <- use your latest folder
plot_dir = os.path.join(run_dir, "plot")

traj_jsons = sorted(glob.glob(os.path.join(plot_dir, "trj_*.json")))
print("Found:", traj_jsons)

latest_json = traj_jsons[-1]
print("Using final trajectory file:", latest_json)

Found: ['/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_0023.json', '/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_0052.json', '/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_0075.json', '/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_0099.json', '/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_0120.json', '/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_0141.json', '/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_0160.json', '/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_0181.json', '/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_0196.json', '/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_0216.json', '/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_0237.json', '/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_0262.json', '/conten

In [37]:
with open(latest_json, "r") as f:
    data = json.load(f)

print("Keys:", data.keys())
print("First trajectory entry:", data["trj_est"][0])
print("Length:", len(data["trj_est"]))

Keys: dict_keys(['trj_id', 'trj_est', 'trj_gt'])
First trajectory entry: [[0.8565045907016036, 0.3657903301947712, -0.3641391498357176, 1.3013000000000001], [0.515472895780805, -0.5703826028174923, 0.639493064949491, 0.8587000000000004], [0.02622174331409512, -0.7354326078840551, -0.6770903185236271, 1.5166], [0.0, 0.0, 0.0, 1.0]]
Length: 138


In [38]:
import os, json, numpy as np

run_dir = "/content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53"  # <-- your latest run
plot_dir = os.path.join(run_dir, "plot")

# pick last trj_XXXX.json as "final"
traj_files = sorted(f for f in os.listdir(plot_dir) if f.startswith("trj_") and f.endswith(".json"))
print("Found traj files:", traj_files)
latest_json = os.path.join(plot_dir, traj_files[-1])
print("Using:", latest_json)

with open(latest_json, "r") as f:
    data = json.load(f)

trj_id  = data["trj_id"]   # timestamps (or frame ids)
trj_est = data["trj_est"]  # list of 4x4 pose matrices

print("Num poses:", len(trj_est), " Num ids:", len(trj_id))
print("First pose shape (rows, cols):", len(trj_est[0]), "x", len(trj_est[0][0]))

Found traj files: ['trj_0023.json', 'trj_0052.json', 'trj_0075.json', 'trj_0099.json', 'trj_0120.json', 'trj_0141.json', 'trj_0160.json', 'trj_0181.json', 'trj_0196.json', 'trj_0216.json', 'trj_0237.json', 'trj_0262.json', 'trj_0282.json', 'trj_0297.json', 'trj_0312.json', 'trj_0327.json', 'trj_0350.json', 'trj_0368.json', 'trj_0383.json', 'trj_0399.json', 'trj_0423.json', 'trj_0449.json', 'trj_0474.json', 'trj_0494.json', 'trj_0509.json', 'trj_0524.json', 'trj_0551.json', 'trj_final.json']
Using: /content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_final.json
Num poses: 138  Num ids: 138
First pose shape (rows, cols): 4 x 4


In [39]:
def rotmat_to_quat(R):
    """
    Convert 3x3 rotation matrix to quaternion (qx, qy, qz, qw)
    using a standard numerically-stable method.
    """
    R = np.array(R, dtype=float)
    t = np.trace(R)
    if t > 0:
        S = np.sqrt(t + 1.0) * 2.0
        qw = 0.25 * S
        qx = (R[2,1] - R[1,2]) / S
        qy = (R[0,2] - R[2,0]) / S
        qz = (R[1,0] - R[0,1]) / S
    elif (R[0,0] > R[1,1]) and (R[0,0] > R[2,2]):
        S = np.sqrt(1.0 + R[0,0] - R[1,1] - R[2,2]) * 2.0
        qw = (R[2,1] - R[1,2]) / S
        qx = 0.25 * S
        qy = (R[0,1] + R[1,0]) / S
        qz = (R[0,2] + R[2,0]) / S
    elif R[1,1] > R[2,2]:
        S = np.sqrt(1.0 + R[1,1] - R[0,0] - R[2,2]) * 2.0
        qw = (R[0,2] - R[2,0]) / S
        qx = (R[0,1] + R[1,0]) / S
        qy = 0.25 * S
        qz = (R[1,2] + R[2,1]) / S
    else:
        S = np.sqrt(1.0 + R[2,2] - R[0,0] - R[1,1]) * 2.0
        qw = (R[1,0] - R[0,1]) / S
        qx = (R[0,2] + R[2,0]) / S
        qy = (R[1,2] + R[2,1]) / S
        qz = 0.25 * S
    return qx, qy, qz, qw

tum_path = os.path.join(run_dir, "traj_est_tum.txt")

with open(tum_path, "w") as f:
    for t, pose in zip(trj_id, trj_est):
        M = np.array(pose, dtype=float)      # 4x4
        R = M[:3, :3]
        trans = M[:3, 3]
        qx, qy, qz, qw = rotmat_to_quat(R)

        timestamp = float(t)  # if these are frame indices, it's still fine for evo
        f.write(
            f"{timestamp:.9f} {trans[0]} {trans[1]} {trans[2]} "
            f"{qx} {qy} {qz} {qw}\n"
        )

print("Wrote TUM trajectory to:", tum_path)

Wrote TUM trajectory to: /content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/traj_est_tum.txt


In [43]:
import json
import numpy as np
from pathlib import Path

# If scipy isn't installed yet:
# !pip install -q scipy

from scipy.spatial.transform import Rotation as R

# ---- paths ----
results_dir = Path("/content/MonoGS/results/MonoGS_datasets")  # adjust if needed
json_path   = results_dir / "2025-12-03-23-34-53" / "plot" / "trj_final.json"  # use your actual folder name

tum_est_out = results_dir / "traj_est_from_json.txt"
tum_gt_out  = results_dir / "traj_gt_from_json.txt"

print("Using traj file:", json_path)

# ---- load json ----
with open(json_path, "r") as f:
    data = json.load(f)

ts = np.array(data["trj_id"], dtype=float)        # shape (N,)
est_list = np.array(data["trj_est"])              # shape (N, 4, 4)
gt_list  = np.array(data["trj_gt"])               # shape (N, 4, 4)

print("Num estimated poses:", est_list.shape[0])
print("Num GT poses:", gt_list.shape[0])

# Sanity check: they should all be 4x4
assert est_list.shape[1:] == (4, 4)
assert gt_list.shape[1:] == (4, 4)

def pose_to_tum_line(t, pose_4x4):
    """Convert 4x4 homogeneous pose to TUM line: t tx ty tz qx qy qz qw"""
    pose = np.asarray(pose_4x4)
    R_mat = pose[:3, :3]
    t_vec = pose[:3, 3]

    # scipy returns quaternions as [qx, qy, qz, qw]
    qx, qy, qz, qw = R.from_matrix(R_mat).as_quat()
    return f"{t:.9f} {t_vec[0]:.9f} {t_vec[1]:.9f} {t_vec[2]:.9f} {qx:.9f} {qy:.9f} {qz:.9f} {qw:.9f}\n"

# ---- write TUM-style files ----
with open(tum_est_out, "w") as fe, open(tum_gt_out, "w") as fg:
    for t, est_pose, gt_pose in zip(ts, est_list, gt_list):
        fe.write(pose_to_tum_line(t, est_pose))
        fg.write(pose_to_tum_line(t, gt_pose))

print("Wrote:")
print("  est:", tum_est_out)
print("  gt :", tum_gt_out)


Using traj file: /content/MonoGS/results/MonoGS_datasets/2025-12-03-23-34-53/plot/trj_final.json
Num estimated poses: 138
Num GT poses: 138
Wrote:
  est: /content/MonoGS/results/MonoGS_datasets/traj_est_from_json.txt
  gt : /content/MonoGS/results/MonoGS_datasets/traj_gt_from_json.txt


In [47]:
!evo_ape tum \
  /content/MonoGS/results/MonoGS_datasets/traj_gt_from_json.txt \
  /content/MonoGS/results/MonoGS_datasets/traj_est_from_json.txt \
  --align --correct_scale \
  -va

--------------------------------------------------------------------------------
Loaded 138 stamps and poses from: /content/MonoGS/results/MonoGS_datasets/traj_gt_from_json.txt
Loaded 138 stamps and poses from: /content/MonoGS/results/MonoGS_datasets/traj_est_from_json.txt
Synchronizing trajectories...
Found 138 of max. 138 possible matching timestamps between...
	/content/MonoGS/results/MonoGS_datasets/traj_gt_from_json.txt
and:	/content/MonoGS/results/MonoGS_datasets/traj_est_from_json.txt
..with max. time diff.: 0.01 (s) and time offset: 0.0 (s).
--------------------------------------------------------------------------------
Aligning using Umeyama's method... (with scale correction)
Rotation of alignment:
[[ 0.99976911 -0.01751148  0.01245257]
 [ 0.01734242  0.99975769  0.01355705]
 [-0.01268695 -0.01333796  0.99983056]]
Translation of alignment:
[ 0.0214537  -0.06092113  0.03819829]
Scale correction: 0.9989721347325928
--------------------------------------------------------------

In [45]:
!evo_rpe tum \
  /content/MonoGS/results/MonoGS_datasets/traj_gt_from_json.txt \
  /content/MonoGS/results/MonoGS_datasets/traj_est_from_json.txt \
  --align --correct_scale \
  --delta 1 --delta_unit f \
  -va

--------------------------------------------------------------------------------
Loaded 138 stamps and poses from: /content/MonoGS/results/MonoGS_datasets/traj_gt_from_json.txt
Loaded 138 stamps and poses from: /content/MonoGS/results/MonoGS_datasets/traj_est_from_json.txt
Synchronizing trajectories...
Found 138 of max. 138 possible matching timestamps between...
	/content/MonoGS/results/MonoGS_datasets/traj_gt_from_json.txt
and:	/content/MonoGS/results/MonoGS_datasets/traj_est_from_json.txt
..with max. time diff.: 0.01 (s) and time offset: 0.0 (s).
--------------------------------------------------------------------------------
Aligning using Umeyama's method... (with scale correction)
Rotation of alignment:
[[ 0.99976911 -0.01751148  0.01245257]
 [ 0.01734242  0.99975769  0.01355705]
 [-0.01268695 -0.01333796  0.99983056]]
Translation of alignment:
[ 0.0214537  -0.06092113  0.03819829]
Scale correction: 0.9989721347325928
--------------------------------------------------------------